## Initial Setup:
This step initializes the necessary configuration.

In [ ]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-exec" / "models" / "vgg19"))

In [ ]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging
from datetime import datetime

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration, DatasetVersion
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [ ]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

## Downloading Dataset:
Downloading the datasets. We will work with three datasets: 2-A5T0 (train), 2-A5T2 (val), and 2-A5T4 (test). The dataset order when extracting is always set in the list provided when downloading. Additionally, this code will always download the latest version of the datasets.

In [ ]:
# RID of source dataset, if any.
datasets = [
            '2-A5T0',
            '2-A5T2',
            '2-A5T4',
]

to_be_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    to_be_download.append(ds_dict)
EA.add_term(vc.workflow_type, "VGG19 Model Train", description="A workflow to train VGG19 model")

workflow_rid = EA.create_workflow(
    name="VGG19 Model train",
    workflow_type="VGG19 Model Train",
)

download_assets = True

config = ExecutionConfiguration(
    datasets=to_be_download if download_assets else [],
    workflow=workflow_rid,
    description="Instance of training VGG19 model",
)
    
execution = EA.create_execution(config)

In [ ]:
print(execution)

In [ ]:
ds_bag_train = execution.datasets[0]
ds_bag_val = execution.datasets[1]
ds_bag_test = execution.datasets[2]

In [ ]:
output_dir = execution._working_dir / execution.execution_rid

## Preprocessing:
Crop the images and move them to the designated folder for training, validation, and testing.

In [ ]:
train_image_path_cropped, csv_ds_bag_cropped = EA.create_cropped_images(ds_bag = ds_bag_train, 
                                                                              output_dir = output_dir / "dataset" / "train", 
                                                                              crop_to_eye=True)


validation_image_path_cropped, validation_csv_cropped = EA.create_cropped_images(ds_bag = ds_bag_val,
                                                                                 output_dir = output_dir / "dataset" / "val",
                                                                                 crop_to_eye=True)

test_image_path_cropped, test_csv_cropped = EA.create_cropped_images(ds_bag = ds_bag_test,
                                                                     output_dir = output_dir "dataset" / "test",
                                                                     crop_to_eye =True)

In [ ]:
current_date = datetime.now().strftime("%b_%d_%Y") 
print(current_date)

In [ ]:
asset_output_dir = execution._working_dir / execution.execution_rid / "asset"
asset_output_dir.mkdir(parents=True, exist_ok=True)

## Train and Evaluate:

In [ ]:
from vgg19_diagnosis_train import train_and_evaluate
with execution.execute() as exec:
        predictions_results, metrics_summary, model_save_path, training_history_csv = train_and_evaluate(
            train_path=train_image_path_cropped,
            valid_path=validation_image_path_cropped, 
            test_path=test_image_path_cropped, 
            model_path=asset_output_dir,
            log_path=asset_output_dir,
            eval_path=asset_output_dir,
            model_name = f"VGG19_Model_{ds_bag_train.dataset_rid}_{current_date}"
            )
        print("Execution Results:")
        print(predictions_results, metrics_summary, model_save_path, training_history_csv)


## Evaluate Only:
If you already have a VGG19 model, provide its path here to evaluate it directly.

In [ ]:
model_path = "path/to/your/model.h5"

In [ ]:
from vgg19_diagnosis_train import evaluate_only
with execution.execute() as exec:
        predictions_results, metrics_summary = evaluate_only(
            model_path = model_path, 
            model_name = f"VGG19_Model_{ds_bag_train.dataset_rid}_{current_date}", 
            test_path = test_image_path_cropped, 
            output_dir = asset_output_dir,
        )
        print("Execution Results:")
        print(predictions_results, metrics_summary)

## Upload results:
We now need to set up the paths to upload the files to the catalog. Please note that this process uploads a copy of the files, even when clean_folder=True. The original files will remain on your system in case you wish to revisit them later. However, you should consider deleting them afterward to conserve storage.

Alternatively, you can move the files directly to the corresponding path created by execution.asset_file_path(). This ensures that you are uploading the original files instead of creating a copy.

In [ ]:
#This loop creates copies.
for filename in os.listdir(asset_output_dir):
    file_path = os.path.join(asset_output_dir, filename)
    if os.path.isfile(file_path):
        asset_type = ""
        if filename.endswith(('csv', 'jpg', 'txt')):
            asset_type = "Model_Prediction"
        elif filename.endswith('pth'):
            asset_type = "Diagnosis_Model"
        if asset_type != "":
            path = execution.asset_file_path(
                asset_name="Execution_Asset",
                file_name=file_path,
                asset_types=asset_type
            )

In [ ]:
execution.upload_execution_outputs(clean_folder=True)